In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import numpy as np
import sklearn 
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import catboost
import lightgbm
import xgboost
import json 

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Notes: 




In [3]:
data_test = pd.read_csv('data/dota2_csv/dota2_skill_test.csv',index_col='id')
data_train = pd.read_csv('data/dota2_csv/dota2_skill_train.csv',index_col='id')

abilities = pd.read_csv('data/dota2_csv/dota2_abilities.csv', index_col='ability_id')
heroes = pd.read_csv('data/heroes_v1.csv', index_col='hero_id')
items = pd.read_csv('data/dota2_csv/dota2_items.csv')

In [4]:
records = []
import json
i = 0
with open('data/dota2_json/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        records.append(record)
        i+=1
        if i == 500:
            break

# Items

count of sum items types

In [5]:
train_id_items = []
train_id = []
import json
it = 0
with open('data/dota2_json/dota2_skill_test.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        mid_items = []
        for i in record['item_purchase_log']:
            mid_items.append(i['item_id'])
        train_id_items.append(mid_items)
        train_id.append(record['id'])

KeyboardInterrupt: 

In [6]:
train_items_log = pd.DataFrame(train_id_items, index=train_id)
train_items_qual_count = pd.DataFrame(index=train_id)

In [14]:
z = [~np.isnan(train_items_log.head(1).values)]

In [18]:
sum(z)

37

In [6]:
for train_id in tqdm_notebook(train_items_log.index):
    x = train_items_log[train_items_log.index == train_id].values
    x = x[~np.isnan(x)]
    qual_item = [items[items.item_id == i]['qual'].values[0] for i in x]
    for q_i in ['component', 'rare', 'epic', 'consumable', 'artifact', 'common', 'secret_shop']:
        train_items_qual_count.loc[train_id, 'qual_item_{}_count'.format(q_i)] = qual_item.count(q_i)

In [72]:
# testim = []
# r = records[14]
# print(r['id'])
# for i in r['item_purchase_log']:
#     testim.append(i['item_id'])
# qual_item = [items[items.item_id == i]['qual'].values[0] for i in testim]
# qual_item.count('epic')

_test_

In [7]:
test_id_items = []
test_id = []
import json
it = 0
with open('data/dota2_json/dota2_skill_test.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        mid_items = []
        for i in record['item_purchase_log']:
            mid_items.append(i['item_id'])
        test_id_items.append(mid_items)
        test_id.append(record['id'])
        

In [8]:
test_items_log = pd.DataFrame(test_id_items, index=test_id)
test_items_qual_count = pd.DataFrame(index=test_id)

In [6]:
for test_id in tqdm_notebook(test_items_log.index):
    x = test_items_log[test_items_log.index == test_id].values
    x = x[~np.isnan(x)]
    qual_item = [items[items.item_id == i]['qual'].values[0] for i in x]
    for q_i in ['component', 'rare', 'epic', 'consumable', 'artifact', 'common', 'secret_shop']:
        test_items_qual_count.loc[test_id, 'qual_item_{}_count'.format(q_i)] = qual_item.count(q_i)

#### final_items

final items and sum

In [103]:
train_id_items = []
train_id = []
import json
it = 0
with open('data/dota2_json/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        train_id_items.append(record['final_items'])
        train_id.append(record['id'])

In [104]:
train_final_items_log = pd.DataFrame(train_id_items, index=train_id)
train_final_items_qual_and_cost = pd.DataFrame(index=train_id)

In [105]:
for train_id in tqdm_notebook(train_final_items_log.index):
    x = train_final_items_log[train_final_items_log.index == train_id].values[0]
    qual_item = [items[items.item_id == i]['qual'].values[0] for i in x if i != 0]
    sum_cost_items = sum([items[items.item_id == i]['cost'].values[0] for i in x if i != 0])
    train_final_items_qual_and_cost.loc[train_id, 'final_items_sum'] = sum_cost_items
    for q_i in ['component', 'rare', 'epic', 'consumable', 'artifact', 'common', 'secret_shop']:
        train_final_items_qual_and_cost.loc[train_id, 'final_items_{}'.format(q_i)] = qual_item.count(q_i)

In [107]:
train_final_items_qual_and_cost.to_csv('data/train_final_items_qual_and_cost.csv')

In [102]:
# print(records[44]['id'])
# for i in records[44]['final_items']:
#     if i != 0:
#         print(items[items.item_id == i]['qual'].values[0])
#         print(items[items.item_id == i]['cost'].values[0])
#         print("---")

_test_

In [110]:
test_id_items = []
test_id = []
import json
it = 0
with open('data/dota2_json/dota2_skill_test.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        test_id_items.append(record['final_items'])
        test_id.append(record['id'])

In [111]:
test_final_items_log = pd.DataFrame(test_id_items, index=test_id)
test_final_items_qual_and_cost = pd.DataFrame(index=test_id)

In [112]:
for test_id in tqdm_notebook(test_final_items_log.index):
    x = test_final_items_log[test_final_items_log.index == test_id].values[0]
    qual_item = [items[items.item_id == i]['qual'].values[0] for i in x if i != 0]
    sum_cost_items = sum([items[items.item_id == i]['cost'].values[0] for i in x if i != 0])
    test_final_items_qual_and_cost.loc[test_id, 'final_items_sum'] = sum_cost_items
    for q_i in ['component', 'rare', 'epic', 'consumable', 'artifact', 'common', 'secret_shop']:
        test_final_items_qual_and_cost.loc[test_id, 'final_items_{}'.format(q_i)] = qual_item.count(q_i)

In [127]:
test_final_items_qual_and_cost.to_csv('data/test_final_items_qual_and_cost.csv')

#### level up speed

think about different solution

In [197]:
train_id_level_up_speed = []
train_id = []
import json
it = 0
with open('data/dota2_json/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        sdvig = []
        for i in range(len(record['level_up_times'])):
            if i + 1 != len(record['level_up_times']):
                sdvig.append(record['level_up_times'][i+1])
        delta_time = []
        
        for q, a in zip(record['level_up_times'], sdvig):
            delta_time.append(a-q)  
            
        if len(delta_time) != 0:
            mean_time = sum(delta_time) / len(delta_time)   
        else:
            mean_time = 0
        
        if mean_time != 0:
            speed = records[ind]['level'] / mean_time
        else:
            speed = 0
            
        train_id.append(record['id'])
        train_id_level_up_speed.append(speed)

In [202]:
train_level_up = pd.DataFrame(train_id_level_up_speed, index=train_id)
train_level_up.columns = ['level_up_speed']
train_level_up.to_csv('data/train_level_up.csv')

_test_

In [216]:
test_id_level_up_speed = []
test_id = []
import json
it = 0
with open('data/dota2_json/dota2_skill_test.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        sdvig = []
        for i in range(len(record['level_up_times'])):
            if i + 1 != len(record['level_up_times']):
                sdvig.append(record['level_up_times'][i+1])
        delta_time = []
        
        for q, a in zip(record['level_up_times'], sdvig):
            delta_time.append(a-q)  
            
        if len(delta_time) != 0:
            mean_time = sum(delta_time) / len(delta_time)   
        else:
            mean_time = 0
        
        if mean_time != 0:
            speed = records[ind]['level'] / mean_time
        else:
            speed = 0
            
        test_id.append(record['id'])
        test_id_level_up_speed.append(speed)

In [217]:
test_level_up = pd.DataFrame(test_id_level_up_speed, index=test_id)
test_level_up.columns = ['level_up_speed']
test_level_up.to_csv('data/test_level_up.csv')

#### damage_targets

kinds of damage

    + make binary of important actions

In [2]:
train_id_damage_targets = []
train_id = []
import json
it = 0
with open('data/dota2_json/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        mid_ar = []
        for i in record['damage_targets'].items():
            mid_ar.append(i)
            
        train_id_damage_targets.append(mid_ar)
        train_id.append(record['id'])

In [3]:
train_id_damage_targets_log = pd.DataFrame(train_id_damage_targets, index=train_id)
train_id_damage_targets = pd.DataFrame(index=train_id)

In [4]:
for train_id in tqdm_notebook(train_id_damage_targets_log.index):
    for i in train_id_damage_targets_log[train_id_damage_targets_log.index == train_id].values[0]:
        if i != None:
            train_id_damage_targets.loc[train_id, '{}'.format(i[0])] = i[1]

In [5]:
train_id_damage_targets.fillna(0).to_csv('data/train_id_damage_targets.csv')

_test_

In [6]:
test_id_damage_targets = []
test_id = []
import json
it = 0
with open('data/dota2_json/dota2_skill_test.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        mid_ar = []
        for i in record['damage_targets'].items():
            mid_ar.append(i)
            
        test_id_damage_targets.append(mid_ar)
        test_id.append(record['id'])

In [7]:
test_id_damage_targets_log = pd.DataFrame(test_id_damage_targets, index=test_id)
test_id_damage_targets = pd.DataFrame(index=test_id)

In [8]:
for test_id in tqdm_notebook(test_id_damage_targets_log.index):
    for i in test_id_damage_targets_log[test_id_damage_targets_log.index == test_id].values[0]:
        if i != None:
            test_id_damage_targets.loc[test_id, '{}'.format(i[0])] = i[1]

In [9]:
test_id_damage_targets.fillna(0).to_csv('data/test_id_damage_targets.csv')

In [ ]:
test_id_damage_targets[test_id_damage_targets.]

<hr>

#### series

count delte between sum, mean, std player team and player 

_train_

In [60]:
train_series = []
train_id = []
import json
it = 0
with open('data/dota2_json/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        p = np.array(record['series']['player_gold'])
        p_sum = p.sum()
        p_mean = p.mean()
        p_std = p.std()
        
        t = np.array(record['series']['{}_gold'.format(record['player_team'])])
        t_sum = t.sum()
        t_mean = t.mean()
        t_std = t.std()
        train_id.append(record['id'])
        
        delta_sum =  t_sum - p_sum
        delta_mean = t_mean - p_mean
        delta_std = t_std - p_std
        
        train_series.append([p_sum, p_mean, p_std, t_sum, t_mean, t_std, delta_sum, delta_mean, delta_std])

In [61]:
train_series = pd.DataFrame(train_series, index=train_id)
train_series.columns = ['p_sum','p_mean','p_std','t_sum','t_mean','t_std', 'delta_sum', 'delta_mean', 'delta_std']
train_series.fillna(0.1).to_csv('data/train_series.csv')

_test_

In [89]:
test_series = []
test_id = []
import json
it = 0
with open('data/dota2_json/dota2_skill_test.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        p = np.array(record['series']['player_gold'])
        p_sum = p.sum()
        p_mean = p.mean()
        p_std = p.std()
        
        t = np.array(record['series']['{}_gold'.format(record['player_team'])])
        t_sum = t.sum()
        t_mean = t.mean()
        t_std = t.std()
        test_id.append(record['id'])
        
        delta_sum =  t_sum - p_sum
        delta_mean = t_mean - p_mean
        delta_std = t_std - p_std
        
        test_series.append([p_sum, p_mean, p_std, t_sum, t_mean, t_std, delta_sum, delta_mean, delta_std])

In [90]:
test_series = pd.DataFrame(test_series, index=test_id)
test_series.columns = ['p_sum','p_mean','p_std','t_sum','t_mean','t_std', 'delta_sum', 'delta_mean', 'delta_std']
test_series.fillna(0.1).to_csv('data/test_series.csv')

<hr>

#### teammates

_train_

In [126]:
from sklearn.preprocessing import OneHotEncoder

In [122]:
train_teammates = []
train_id = []
import json
it = 0
with open('data/dota2_json/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        train_id.append(record['id'])
        train_teammates.append(record['{}_heroes'.format(record['player_team'])])

In [123]:
train_teammates_id = pd.DataFrame(train_teammates, index=train_id)
train_teammates= pd.DataFrame(index=train_id)

In [199]:
h0 = OneHotEncoder().fit_transform(train_teammates_id[0].values.reshape(-1,1))
h1 = OneHotEncoder().fit_transform(train_teammates_id[1].values.reshape(-1,1))
h2 = OneHotEncoder().fit_transform(train_teammates_id[2].values.reshape(-1,1))
h3 = OneHotEncoder().fit_transform(train_teammates_id[3].values.reshape(-1,1))
h4 = OneHotEncoder().fit_transform(train_teammates_id[4].values.reshape(-1,1))

In [200]:
df_h0 = pd.DataFrame(h0.toarray(), index=train_id)
df_h1 = pd.DataFrame(h1.toarray(), index=train_id)
df_h2 = pd.DataFrame(h2.toarray(), index=train_id)
df_h3 = pd.DataFrame(h3.toarray(), index=train_id)
df_h4 = pd.DataFrame(h4.toarray(), index=train_id)

In [201]:
train_teammates = df_h0 + df_h1 + df_h2 + df_h3 + df_h4

In [206]:
train_teammates = train_teammates.astype('int')

In [207]:
train_teammates.to_csv('data/train_teammates.csv')

_test_

In [208]:
test_teammates = []
test_id = []
import json
it = 0
with open('data/dota2_json/dota2_skill_test.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        test_id.append(record['id'])
        test_teammates.append(record['{}_heroes'.format(record['player_team'])])

In [209]:
test_teammates_id = pd.DataFrame(test_teammates, index=test_id)
test_teammates= pd.DataFrame(index=test_id)

In [210]:
h0 = OneHotEncoder().fit_transform(test_teammates_id[0].values.reshape(-1,1))
h1 = OneHotEncoder().fit_transform(test_teammates_id[1].values.reshape(-1,1))
h2 = OneHotEncoder().fit_transform(test_teammates_id[2].values.reshape(-1,1))
h3 = OneHotEncoder().fit_transform(test_teammates_id[3].values.reshape(-1,1))
h4 = OneHotEncoder().fit_transform(test_teammates_id[4].values.reshape(-1,1))

In [212]:
df_h0 = pd.DataFrame(h0.toarray(), index=test_id)
df_h1 = pd.DataFrame(h1.toarray(), index=test_id)
df_h2 = pd.DataFrame(h2.toarray(), index=test_id)
df_h3 = pd.DataFrame(h3.toarray(), index=test_id)
df_h4 = pd.DataFrame(h4.toarray(), index=test_id)

In [213]:
test_teammates = df_h0 + df_h1 + df_h2 + df_h3 + df_h4

In [214]:
test_teammates = test_teammates.astype('int')

In [215]:
test_teammates.to_csv('data/test_teammates.csv')

add mean and winrate by teammates

'agi_gain', 'attack_range', 'attack_rate'

_test_

In [328]:
train_id = []
train_tm_stats = []
for p_id in tqdm_notebook(train_teammates_id.index):
    tm_roles = np.array([0,0,0,0,0,0,0,0,0])
    tm_agi = []
    tm_attack_range = []
    tm_attack_rate = []
    tm_winrate = []
    
    for tm in train_teammates_id[train_teammates_id.index == p_id].values[0]:
        tm_roles += heroes[heroes.index == tm].iloc[:,-9:].values[0]
        tm_agi.append(heroes[heroes.index == tm]['agi_gain'].values[0])
        tm_attack_range.append(heroes[heroes.index == tm]['attack_range'].values[0])
        tm_attack_rate.append(heroes[heroes.index == tm]['attack_rate'].values[0])
        tm_winrate.append(heroes[heroes.index == tm]['winrate'].values[0])
    train_id.append(p_id)

    tm_agi_mean = np.array(tm_agi).mean()
    tm_attack_range_mean = np.array(tm_attack_range).mean()
    tm_attack_rate_mean = np.array(tm_attack_rate).mean()
    tm_winrate_mean = np.array(tm_winrate).mean()
    
    tm_agi_std = np.array(tm_agi).std()
    tm_attack_range_std = np.array(tm_attack_range).std()
    tm_attack_rate_std = np.array(tm_attack_rate).std()
    tm_winrate_std = np.array(tm_winrate).std()
    
    r0 = tm_roles[0]
    r1 = tm_roles[1]
    r2 = tm_roles[2]
    r3 = tm_roles[3]
    r4 = tm_roles[4]
    r5 = tm_roles[5]
    r6 = tm_roles[6]
    r7 = tm_roles[7]
    r8 = tm_roles[8]
    
    train_tm_stats.append([tm_agi_mean, tm_attack_range_mean, tm_attack_rate_mean,
                          tm_winrate_mean, tm_agi_std, tm_attack_range_std, tm_attack_rate_std, tm_winrate_std, r0,r1,r2,r3,r4,r5,r6,r7,r8])

In [329]:
train_tm_stats = pd.DataFrame(train_tm_stats, index= train_id)
train_tm_stats.to_csv('data/train_tm_stats.csv')

_test_

In [333]:
test_id = []
test_tm_stats = []
for p_id in tqdm_notebook(test_teammates_id.index):
    tm_roles = np.array([0,0,0,0,0,0,0,0,0])
    tm_agi = []
    tm_attack_range = []
    tm_attack_rate = []
    tm_winrate = []
    
    for tm in test_teammates_id[test_teammates_id.index == p_id].values[0]:
        tm_roles += heroes[heroes.index == tm].iloc[:,-9:].values[0]
        tm_agi.append(heroes[heroes.index == tm]['agi_gain'].values[0])
        tm_attack_range.append(heroes[heroes.index == tm]['attack_range'].values[0])
        tm_attack_rate.append(heroes[heroes.index == tm]['attack_rate'].values[0])
        tm_winrate.append(heroes[heroes.index == tm]['winrate'].values[0])
    test_id.append(p_id)

    tm_agi_mean = np.array(tm_agi).mean()
    tm_attack_range_mean = np.array(tm_attack_range).mean()
    tm_attack_rate_mean = np.array(tm_attack_rate).mean()
    tm_winrate_mean = np.array(tm_winrate).mean()
    
    tm_agi_std = np.array(tm_agi).std()
    tm_attack_range_std = np.array(tm_attack_range).std()
    tm_attack_rate_std = np.array(tm_attack_rate).std()
    tm_winrate_std = np.array(tm_winrate).std()
    
    r0 = tm_roles[0]
    r1 = tm_roles[1]
    r2 = tm_roles[2]
    r3 = tm_roles[3]
    r4 = tm_roles[4]
    r5 = tm_roles[5]
    r6 = tm_roles[6]
    r7 = tm_roles[7]
    r8 = tm_roles[8]
    
    test_tm_stats.append([tm_agi_mean, tm_attack_range_mean, tm_attack_rate_mean,
                          tm_winrate_mean, tm_agi_std, tm_attack_range_std, tm_attack_rate_std, tm_winrate_std, r0,r1,r2,r3,r4,r5,r6,r7,r8])

In [335]:
test_tm_stats = pd.DataFrame(test_tm_stats, index= test_id)

In [336]:
test_tm_stats.to_csv('data/test_tm_stats.csv')

# Player in team 

In [46]:
cols = ['skilled','gold_per_min','xp_per_min', 'avg_kills_x16','avg_deaths_x16','avg_assists_x16','avg_gpm_x16','avg_xpm_x16','best_kills_x16','best_assists_x16','best_gpm_x16','best_xpm_x16','win_streak','best_win_streak']
mn_1 =data_train.groupby('hero_id')[cols].mean()

In [132]:
train_deltas = []
train_id = []
import json
it = 0
with open('data/dota2_json/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        l_skilled = [] 
        l_gold_per_min =  []
        l_xp_per_min =  []
        l_avg_kills_x16 =  []
        l_avg_deaths_x16 =  []
        l_avg_assists_x16 =  []
        l_avg_gpm_x16 =  []
        l_avg_xpm_x16 =  []
        l_best_kills_x16 =  []
        l_best_assists_x16 =  []
        l_best_gpm_x16 =  []
        l_best_xpm_x16 =  []
        l_win_streak =  []
        l_best_win_streak =  []

        for players in record['{}_heroes'.format(record['player_team'])]:
            if players == record['hero_id']:  

                p_skilled            = mn_1.at[players, 'skilled'] 
                p_gold_per_min       = mn_1.at[players, 'gold_per_min']
                p_xp_per_min         = mn_1.at[players, 'xp_per_min']   
                p_avg_kills_x16      = mn_1.at[players, 'avg_kills_x16']
                p_avg_deaths_x16     = mn_1.at[players, 'avg_deaths_x16']
                p_avg_assists_x16    = mn_1.at[players, 'avg_assists_x16']
                p_avg_gpm_x16        = mn_1.at[players, 'avg_gpm_x16']
                p_avg_xpm_x16        = mn_1.at[players, 'avg_xpm_x16']
                p_best_kills_x16     = mn_1.at[players, 'best_kills_x16']
                p_best_assists_x16   = mn_1.at[players, 'best_assists_x16']
                p_best_gpm_x16       = mn_1.at[players, 'best_gpm_x16']
                p_best_xpm_x16       = mn_1.at[players, 'best_xpm_x16']
                p_win_streak         = mn_1.at[players, 'win_streak']
                p_best_win_streak    = mn_1.at[players, 'best_win_streak']

                l_skilled.append(p_skilled)
                l_gold_per_min.append(p_gold_per_min)
                l_xp_per_min.append(p_xp_per_min)
                l_avg_kills_x16.append(p_avg_kills_x16)
                l_avg_deaths_x16.append(p_avg_deaths_x16) 
                l_avg_assists_x16.append(p_avg_assists_x16)  
                l_avg_gpm_x16.append(p_avg_gpm_x16)
                l_avg_xpm_x16.append(p_avg_xpm_x16)
                l_best_kills_x16.append(p_best_kills_x16) 
                l_best_assists_x16.append(p_best_assists_x16)   
                l_best_gpm_x16.append(p_best_gpm_x16)
                l_best_xpm_x16.append(p_best_xpm_x16)
                l_win_streak.append(p_win_streak)
                l_best_win_streak.append(p_best_win_streak)  

            l_skilled.append(mn_1.at[players, 'skilled']) 
            l_gold_per_min.append(mn_1.at[players, 'gold_per_min'])
            l_xp_per_min.append(mn_1.at[players, 'xp_per_min'])
            l_avg_kills_x16.append(mn_1.at[players, 'avg_kills_x16'])
            l_avg_deaths_x16.append(mn_1.at[players, 'avg_deaths_x16'])
            l_avg_assists_x16.append(mn_1.at[players, 'avg_assists_x16'])
            l_avg_gpm_x16.append(mn_1.at[players, 'avg_gpm_x16'])
            l_avg_xpm_x16.append(mn_1.at[players, 'avg_xpm_x16'])
            l_best_kills_x16.append(mn_1.at[players, 'best_kills_x16'])
            l_best_assists_x16.append(mn_1.at[players, 'best_assists_x16'])  
            l_best_gpm_x16.append(mn_1.at[players, 'best_gpm_x16'])
            l_best_xpm_x16.append(mn_1.at[players, 'best_xpm_x16'])
            l_win_streak.append(mn_1.at[players, 'win_streak'])
            l_best_win_streak.append(mn_1.at[players, 'best_win_streak']) 

        delta_skilled = max(l_skilled) - p_skilled 
        delta_gold_per_min =  max(l_gold_per_min) - p_gold_per_min
        delta_xp_per_min = max(l_xp_per_min) - p_xp_per_min
        delta_avg_kills_x16 = max(l_avg_kills_x16) - p_avg_kills_x16
        delta_avg_deaths_x16 = max(l_avg_deaths_x16) - p_avg_deaths_x16
        delta_avg_assists_x16 =  max(l_avg_assists_x16) -  p_avg_assists_x16
        delta_avg_gpm_x16 = max(l_avg_gpm_x16) - p_avg_gpm_x16  
        delta_avg_xpm_x16 =  max(l_avg_xpm_x16) - p_avg_xpm_x16
        delta_best_kills_x16 = max(l_best_kills_x16) - p_best_kills_x16
        delta_best_assists_x16 = max(l_best_assists_x16) - p_best_assists_x16
        delta_best_gpm_x16 = max(l_best_gpm_x16) - p_best_gpm_x16
        delta_best_xpm_x16 = max(l_best_xpm_x16) - p_best_xpm_x16
        delta_win_streak = max(l_win_streak) - p_win_streak
        delta_best_win_streak = max(l_best_win_streak) -  p_best_win_streak
        
        train_deltas.append(  [delta_skilled, delta_gold_per_min, delta_xp_per_min, delta_avg_kills_x16 , delta_avg_deaths_x16 , delta_avg_assists_x16 , delta_avg_gpm_x16, delta_avg_xpm_x16, delta_best_kills_x16 , delta_best_assists_x16, delta_best_gpm_x16, delta_best_xpm_x16, delta_win_streak, delta_best_win_streak])
        train_id.append(record['id'])

In [133]:
df = pd.DataFrame(train_deltas, index=train_id)

In [134]:
df.to_csv('data/deltas_p_team.csv')